In [53]:
import os
import json 
import logging
from datetime import datetime

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri # get the Docker image which contains the image classif agorithm

## explicacion de codi


sadinfoaksndf

sfldansfodknsa


In [54]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [55]:
session = sagemaker.Session()
region = session.boto_region_name

In [56]:
arn_role = sagemaker.get_execution_role()

In [57]:
bucket_name = "smart-factory-sagemaker"
data_path = "stock-inference"

In [58]:
nclasses = 3
nimgs_train = 16
nepochs = 100
mini_batch_size = 6

In [59]:
train_instance = "ml.g4dn.xlarge"
job_name_prefix = "smart-factory-wh-stock-inference"

In [60]:
train_image_uri = sagemaker.image_uris.retrieve(
    #### TRY HUGGING FACE AFTER ###
    ## IF THE FRAMEWORK IS 3RD PARTY. THE IMAGE SCOPE IS DIFFERENT FOR TRAINING AND INFERENCE ##
    framework="image-classification",
    region=region,
    image_scope="training"
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [61]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")

In [62]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=arn_role,
    instance_count=1,
    instance_type=train_instance,
    volume_size=30,
    max_run=3600,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=session
)

In [63]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,
    epochs=nepochs,
    image_shape="3,240,320",    
    num_training_samples=nimgs_train,
    mini_batch_size=nimgs_train,
    num_layers=18,
    use_pretrained_model=1,
    multi_label=1
)

In [64]:
s3_training_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_training_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")


In [65]:
train_imgs = sagemaker.inputs.TrainingInput(
    s3_training_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix"
)

valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix"
)

train_annot = sagemaker.inputs.TrainingInput(
    s3_training_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix"
)

valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix"
)


data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot
}

In [67]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + "-" +timestamp
print(job_name)


# Run the line below to train the model: 
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: smart-factory-wh-stock-inference-2023-05-15-11-57-11


smart-factory-wh-stock-inference-2023-05-15-11-57-11
2023-05-15 11:57:11 Starting - Starting the training job...
2023-05-15 11:57:26 Starting - Preparing the instances for training......
2023-05-15 11:58:23 Downloading - Downloading input data...
2023-05-15 11:58:43 Training - Downloading the training image............
2023-05-15 12:00:44 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Mon May 15 12:00:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usa

In [150]:
infer_instance_type = "ml.t2.medium"
model_name = job_name
endpoint_name = "smart-factory-wh-stock-inference"

In [151]:
clf_predictor = clf_estimator.deploy(
    initial_instance_count=1,
    instance_type=infer_instance_type,
    endpoint_name=endpoint_name,
    model_name=model_name
)

INFO:sagemaker:Creating model with name: smart-factory-wh-stock-inference-2023-05-07-14-51-38
INFO:sagemaker:Creating endpoint-config with name smart-factory-wh-stock-inference
INFO:sagemaker:Creating endpoint with name smart-factory-wh-stock-inference


-----------------------!

In [ ]:
sgmkr_runtime = boto3.client("runtime.sagemaker")